# 自动阅卷

初中历史主观题作答数据集

In [1]:
import re

In [38]:
import jieba
import jieba.analyse

In [3]:
import pandas as pd

In [4]:
from functools import reduce

## Configure

### Re

In [5]:
# Chinese punctuation
re_cn_punc = re.compile(r'[\uff1a\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b:;,.]')

In [6]:
re_cn_score = re.compile(f'\u5404\d+\u5206')

In [7]:
re_slash = re.compile(r'(?!=\d+)-(?!=\d+)')

In [8]:
# question id decode
re_q_decode = re.compile(r'(\d+)(?:\D+(\d+))?')

In [9]:
re_whitespace = re.compile(r'\s+|_x000D_')

In [10]:
re_p_score = re.compile(r"(?:（|\()(?: *)([0-9.]+)分(?: *)(?:）|\))")

In [11]:
# Sub Serial number
re_serial = re.compile(r"(?:[\u2460-\u2468])|(?:（|\()(?: *)([0-9.]+)(?: *)(?:）|\))")

### Jieba

In [12]:
stop_word_path = r"cn_stopwords.txt"
stop_words = open(stop_word_path,'r',encoding='utf-8').read().split('\n')+['\n']

## Clean Data

In [13]:
data_1 = pd.read_excel('data.xlsx', sheet_name=0)
data_2 = pd.read_excel('data.xlsx', sheet_name=1)
data_3 = pd.read_excel('data.xlsx', sheet_name=2)

In [14]:
question_data = data_1.loc[:, ['题目id（唯一）', '题目文本', '题目分值', '该题标准答案（含得分点）']]

In [15]:
student_data = data_2.loc[:, ['学生id', '题目id', '该生作答文本', '该生作答得分', '该生作答得分点']]

In [16]:
question_data = question_data[~reduce(
    lambda x, y: x[1] | y[1],
    question_data.isna().iteritems()
)]

In [17]:
student_data = student_data[~reduce(
    lambda x, y: x[1] | y[1],
    student_data.isna().iteritems()
)]

In [18]:
question = pd.DataFrame({
    'id': question_data['题目id（唯一）'].apply(
        lambda x: '{0[0]:0>2s}{0[1]:0>2s}'.format([
            item if item else '0'
            for item in re_q_decode.search(x).groups()
        ])
    ).astype('string'),
    'text': question_data['题目文本'].apply(
        lambda x: re_whitespace.sub('', x)
    ).astype('string'),
    'score': question_data['题目分值'].astype(float),
    'answer': question_data['该题标准答案（含得分点）'].apply(
       lambda x: re_whitespace.sub('', x)
    ).astype('string')
})
question = question.reset_index(drop=True)

In [19]:
question.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      151 non-null    string 
 1   text    151 non-null    string 
 2   score   151 non-null    float64
 3   answer  151 non-null    string 
dtypes: float64(1), string(3)
memory usage: 4.8 KB


In [20]:
student = pd.DataFrame({
    'id': student_data['学生id'].apply(
        lambda x: int(x) if not pd.isna(x) else '202020000'
    ).astype(int),
    'q_id': student_data['题目id'].apply(
        lambda x: '{0[0]:0>2s}{0[1]:0>2s}'.format([
            item if item else '0'
            for item in re_q_decode.search(x).groups()
        ])
    ).astype('string'),
    'answer': student_data['该生作答文本'].apply(
       lambda x: re_whitespace.sub('', x)
    ).astype('string'),
    'score': student_data['该生作答得分'].astype(float),
    'point': student_data['该生作答得分点'].apply(
       lambda x: re_whitespace.sub('', x)
    ).astype('string')
})
student = student.reset_index(drop=True)

In [21]:
student.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      151 non-null    int32  
 1   q_id    151 non-null    string 
 2   answer  151 non-null    string 
 3   score   151 non-null    float64
 4   point   151 non-null    string 
dtypes: float64(1), int32(1), string(3)
memory usage: 5.4 KB


In [22]:
question

,id,text,score,answer
0,1100,为推翻北洋军阀的反动通知和抗击日本侵略、挽救民族危亡、国共两党曾两次合作。请分别说出国共两次...,4.0,第一次国共合作，进行国民革命运动，有力推动了北伐战争的胜利，推翻北洋军阀统治（2分）；第二次...
1,1701,列举孙中山领导辛亥革命的指导思想,4.0,三民主义（1分）；民族（1分）；民权（1分）；民生（1分）。
2,1702,列举第一，二次工业革命中出现的新式交通工具,4.0,轮船（1分）；火车（1分）；汽车（1分）；飞机（1分）。
3,1703,列举《南京条约》《马关条约》中被列强割占的地方？,6.0,《南京条约》，英国（1分）；香港岛（1分）；《马关条约》，日本（1分）；辽东半岛（1分）；台...
4,0201,对“唐太宗与贞观之治的评价”,6.0,（1）知人善任（1分），他用人不计较资历出身，不计亲疏恩怨，凡有治国安邦之才，都委以重任。（...
...,...,...,...,...
146,1908,唐太宗统治时期出现了怎样的政治、经济局面?,2.0,政治局面:①政治清明;②社会秩序相对安定;③阶级矛盾相对缓和;④民族团结。(答出任意:一点即...
147,1909,第一个与新中国建交的是哪一国家?中国长期以来奉行怎样的外交政策以及外交理念?,3.0,国家:苏联。(1分)政策:独立自主的和平外交政策。(1分)理念:共商共建共享的全球治理观，构...
148,2001,美国独立战争爆发的导火索和原因？,4.0,导火索：波士顿倾茶事件（2）根本原因：为了独立自主，为了反抗英国对殖民地的无休止的压榨。（2）
149,2002,新文化运动的意义,3.0,"动摇了封建思想的统治地位,人们的思想得到空前解放。（0.5分）；民主和科学思想得到弘扬,推动..."


In [23]:
student['answer'][:100]

0     第一次国共合作的形成加快了中国革命前进步伐.合作失败后，使党开始成长，懂得了进行土地革命和掌...
1                                         三民主义：民族，民权，民生
2                                                 飞机，火车
3                    《南京条约》中英国割让香港岛；《马关条约》中日本割让辽东半岛和台湾岛
4                                              民权、民族、民生
                            ...                        
95                    南方温暖湿润，土地肥沃，有利于农业发展经过统治者和老百姓的长期开发
96                                     割香港岛给英国;赔款2100万元
97                                             反帝反封建；爱国
98                              和平共处；互相尊重国家主权；互惠互利；共同发展
99                                            推翻北洋军阀的统治
Name: answer, Length: 100, dtype: string

In [24]:
student.to_csv('student.csv', index=False, encoding='utf-8_sig')
question.to_csv('question.csv', index=False, encoding='utf-8_sig')

## Processing

### Answer Points Table

In [25]:
def get_point_score():
    info_dict = {
        "q_id": list(),
        "p_id": list(),
        "point": list(),
        "p_score": list(),
    }
    temp_point_score = []
    
    for k, item in question['answer'].iteritems():
        ans_split = re_p_score.split(item)
        ans_tuple = [
            (ans_split[i*2], float(ans_split[i*2+1])) 
            for i in range(len(ans_split) // 2)
        ]
        if not ans_tuple:
            ans_tuple = [(item, question['score'][k])]
        temp_point_score.append(ans_tuple)
    
    for k, item in enumerate(temp_point_score):
        for i, point in enumerate(item):
            info_dict["q_id"].append(k)
            info_dict["p_id"].append(i)
            info_dict["point"].append(point[0])
            info_dict["p_score"].append(point[1])

    return pd.DataFrame(info_dict)

In [26]:
answer = get_point_score()

In [27]:
answer

,q_id,p_id,point,p_score
0,0,0,第一次国共合作，进行国民革命运动，有力推动了北伐战争的胜利，推翻北洋军阀统治,2.0
1,0,1,；第二次国共合作，建立抗日民族统一展现，领导全民族抗战，成为抗日胜利的保障,2.0
2,1,0,三民主义,1.0
3,1,1,；民族,1.0
4,1,2,；民权,1.0
...,...,...,...,...
432,149,2,；为五四运动爆发起了思想宣传动员作用。,0.5
433,149,3,"；后期传播的社会主义思想,为中国先进知识分子所接受,作为拯救国家、改造社会的思想武器。",0.5
434,149,4,；有利于文化的普及和繁荣。,0.5
435,149,5,"；对东西方文化存在绝对肯定或绝对否定的偏向,一直影响到后来。",5.0


### Split Point words

#### Standard Answer

In [28]:
def split_point_words(point):
    text = re_serial.sub("\n", point)
    text = re.split("\n", text)
    text = [
        re_cn_score.sub(
            '', re_slash.sub('', temp)
        )
        for sub_item in text
        for temp in re_cn_punc.split(sub_item)
        if sub_item if temp
    ]
    return text

In [29]:
def cut_point(point):
    cut_list = []
    words_list = split_point_words(point)
    
    for item in words_list:
        cut_list.extend(jieba.lcut(item))

#     cut_list = reduce(
#         lambda x,y: x if y in x else x + [y],
#         [[], ] + sorted(cut_list)
#     )
    
    cut_list = list(filter(
        lambda x: x not in stop_words,
        cut_list
    ))

    return cut_list

In [30]:
def decode_words(point):
    words = cut_point(point)
    return '\t'.join(words)

In [31]:
answer['words'] = answer['point'].apply(
    decode_words
).astype('string')
answer = answer.drop('point', axis=1)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\east\AppData\Local\Temp\jieba.cache
Loading model cost 0.896 seconds.
Prefix dict has been built successfully.


In [32]:
answer

,q_id,p_id,p_score,words
0,0,0,2.0,第一次 国共合作 进行 国民革命 运动 有力 推动 北伐战争 胜利 推翻 北洋军阀 统治
1,0,1,2.0,第二次 国共合作 建立 抗日 民族 统一 展现 领导 全 民族 抗战 成为 抗日 胜利 保障
2,1,0,1.0,三民主义
3,1,1,1.0,民族
4,1,2,1.0,民权
...,...,...,...,...
432,149,2,0.5,五四运动 爆发 思想 宣传 动员 作用
433,149,3,0.5,后期 传播 社会主义 思想 中国 先进 知识分子 接受 拯救 国家 改造社会 思想 武器
434,149,4,0.5,有利于 文化 普及 繁荣
435,149,5,5.0,东西方 文化 存在 绝对 肯定 绝对 否定 偏向 一直 影响 后来


In [33]:
answer.to_csv('answer.csv', index=False, encoding='utf-8_sig')

#### Student Answer

In [34]:
student['words'] = student['answer'].apply(
    decode_words
).astype('string')

In [35]:
student = student.drop('answer', axis=1)

In [36]:
student

,id,q_id,score,point,words
0,20210001,1100,2.0,推翻北洋军阀统治（2分）,第一次 国共合作 形成 加快 中国 革命 前进 步伐 合作 失败 使党 成长 懂得 进行 土...
1,202032195,1701,4.0,三民主义（1分）；民族（1分）；民权（1分）；民生（1分）,三民主义 民族 民权 民生
2,202032195,1702,2.0,火车（1分）；飞机（1分）,飞机 火车
3,202032195,1703,5.0,《南京条约》，英国（1分）；香港岛（1分）；《马关条约》，日本（1分）；辽东半岛（1分）；台...,南京 条约 中 英国 割让 香港岛 马关条约 中 日本 割让 辽东半岛 台湾岛
4,202032181,1701,3.0,民族（1分）；民权（1分）；民生（1分）,民权 民族 民生
...,...,...,...,...,...
146,202032141,1908,1.0,社会稳定（1分）,经济 人民 生活 水平 提高 政治 社会 稳定 吏治 清明
147,202032141,1909,3.0,苏联（1分）；独立自主（1分）；共商共建共治（1分）,国家 前 苏联 政策 独立自主 和平 外交政策 理念 共商 共建 共治 全球 治理 观
148,202032184,2001,4.0,波士顿倾茶事件（2）；独立自主（2）,导火索 波士顿 倾茶 事件 原因 独立自主
149,202032184,2002,1.5,动摇了封建思想的统治地位（0.5）；民主和科学思想（0.5）；新思想的传播（0.5）,新文化运动 我国 历史 一次 空前 思想 解放运动 彻底 动摇 封建思想 统治 地位 国民 ...


In [37]:
student.to_csv('student.csv', index=False, encoding='utf-8_sig')

### IDF

In [41]:
jieba.analyse.extract_tags?

Signature:
jieba.analyse.extract_tags(
    sentence,
    topK=20,
    withWeight=False,
    allowPOS=(),
    withFlag=False,
)
Docstring:
Extract keywords from sentence using TF-IDF algorithm.
Parameter:
    - topK: return how many top keywords. `None` for all possible words.
    - withWeight: if True, return a list of (word, weight);
                  if False, return a list of words.
    - allowPOS: the allowed POS list eg. ['ns', 'n', 'vn', 'v','nr'].
                if the POS of w is not in this list,it will be filtered.
    - withFlag: only work with allowPOS is not empty.
                if True, return a list of pair(word, weight) like posseg.cut
                if False, return a list of words
File:      c:\environment\python\python39\lib\site-packages\jieba\analyse\tfidf.py
Type:      method


In [40]:
jieba.analyse.extract_tags(
    "第一次国共合作的形成加快了中国革命前进步伐.合作失败后，使党开始成长，懂得了进行土地革命",
    withWeight=True
)

[('使党', 0.74717296893125),
 ('土地革命', 0.615014665089375),
 ('国共合作', 0.551204580436875),
 ('懂得', 0.437308149649375),
 ('步伐', 0.432770598845625),
 ('前进', 0.421433885483125),
 ('失败', 0.38988811479875),
 ('成长', 0.381307770855625),
 ('加快', 0.364517168338125),
 ('第一次', 0.356877873191875),
 ('革命', 0.3459378292625),
 ('合作', 0.344358011695625),
 ('形成', 0.2670095874675),
 ('开始', 0.2403141942525),
 ('进行', 0.23294529968875),
 ('中国', 0.18920754291625)]